In [2]:
# result: address, holdings, previousTradeTimestamp, boughtTotal, holdingCurrently

import pandas as pd
import alchemyHelper
import graphqlHelper
import json

# GLOBAL VARIABLES
uniswapV2Router = "0x7a250d5630b4cf539739df2c5dacb4c659f2488d"
uniswapV3Router = "0x68b3465833fb72A70ecDF485E0e4C7bD8665Fc45"
weiDivisible = 1000000000000000000

# getting count
with open("mainCOUNT.txt", "r") as f:
    rowNum = int(f.read())
count = rowNum + 1

walletsList = pd.read_csv("wallets_test.csv")
walletsList = walletsList.drop_duplicates()

tokensTraded = 0
stillHoldingOrDusted = 0
sellingIncrementally = 0
accountTransferORNULL = 0
stablecoinTransfer = 0
buysTOTAL = []
sellsTOTAL = []
profits = {}

for i, row in walletsList[rowNum:].iterrows():
    bigDICT = {}
    wallet = row["Address"]
    #print(f"CURRENT WALLET: {wallet}")

    positiveTrade = 0
    negativeTrade = 0

    # BUY SECTION
    buyList = alchemyHelper.getBuys(wallet)
    for dict in buyList:
        txnHash = dict["hash"]
        contract = dict["rawContract"]["address"]

        if contract not in bigDICT:
            bigDICT[contract] = {
                "buys": [],
                "sells": []
            }

        bigDICT[contract]['buys'].append(txnHash)

    # SELL SECTION
    sellList = alchemyHelper.getSells(wallet)
    for dict in sellList:
        txnHash = dict["hash"]
        contract = dict["rawContract"]["address"]

        if contract not in bigDICT:
            bigDICT[contract] = {
                "buys": [],
                "sells": []
            }

        bigDICT[contract]['sells'].append(txnHash)

    # bigDICT[contract]
    # ITERATING BIG DICTIONARY
    for contract in bigDICT:
        # REMOVING DUPLICATES
        bigDICT[contract]["buys"] = [*set(bigDICT[contract]["buys"])]
        bigDICT[contract]["sells"] = [*set(bigDICT[contract]["sells"])]

        # CASE0 - IRREGULARITY
        if (bigDICT[contract]["buys"] == []):
            continue

        # CASE1 - still holding token or rugged/dusted
        if (bigDICT[contract]["sells"] == []):
            stillHoldingOrDusted += 1
            continue

        # CASE2 - Great buy and selling incrementally, smart trader
        buyLen = len(bigDICT[contract]["buys"])
        sellLen = len(bigDICT[contract]["sells"])
        diffLen = sellLen / float(buyLen)
        if (diffLen > 2):
            sellingIncrementally += 1

        excludeThisContract = False
        buys = 0.0
        # CHECKING BUYS GRAPHQL
        for buy in bigDICT[contract]["buys"]:
            #print("TXN: ", buy)
            result = graphqlHelper.getData(buy)
            if (result == -1):
                accountTransferORNULL += 1
                excludeThisContract = True
            elif (result == -2):
                stablecoinTransfer += 1
            else:
                buys += result

        sells = 0.0
        # CHECKING SELLS GRAPHQL
        for sell in bigDICT[contract]["sells"]:
            result = graphqlHelper.getData(sell)
            if (result == -1):
                accountTransferORNULL += 1
                excludeThisContract = True
            elif (result == -2):
                stablecoinTransfer += 1
            else:
                sells += result

        profit = sells - buys
        # STABLECOIN OR JUST TRANSFER
        if (profit == 0.0 or excludeThisContract):
            continue

        # transfer maybe
        if buys == 0: continue

        profit = ((sells - buys) / buys) * 100.0
        
        if profit > 0: positiveTrade += 1
        else: negativeTrade += 1

        buysTOTAL.append(buys)
        sellsTOTAL.append(sells)
        profits[contract] = profit
        tokensTraded += 1

    with open("test.json", "w") as file:
        json.dump(profits, file)

    profitsAVG = sum(profits.values())/len(profits)

    df = pd.DataFrame(columns=list(['Wallet_Address','Tokens_Traded', 'Positive_Trades', 'Negative_Trades', 'Still_Holding_OR_Dusted', 'Sold_Incrementally_(NotJet)', 'Transfer_To_Other_Account_OR_NULL', 'Transfer_Stablecoin', 'Average_Profit']))
    df.loc[count] = [wallet, tokensTraded, positiveTrade, negativeTrade, stillHoldingOrDusted, sellingIncrementally, accountTransferORNULL, stablecoinTransfer, profitsAVG]

    count += 1

df

   


CURRENT WALLET: 0x0678a0cfc4b8e749033e7b452ff19f7bc90d5ad4
TXN:  0xd672bd986d7627671f8f6435cadd7dc397e2fe332cc77ef6af62e11ac6129842
TXN:  0xe5137b017923e94d6c6c3f165603d731f6845d911c41b1f651701e48691b9ad0
TXN:  0x503d4c2160fc561b799cc4c7f7ec027c5721f6e0d113434377d968f5b650aaa7
TXN:  0x8653d46309256926e47b223e961b55259208b3efb1d282195a8119450aed71fe
TXN:  0x5393006b6013efb02089cc7d710e71738d949b0f20133f4fa1e3258355b1eddc
TXN:  0x9b702d2d698f2c1e3e879b50d5a0cf753b57e206681fa6c99f72b7c3cc323c70
TXN:  0xc0f698de3da010b69e7e28cc85c6a09daebed433a029d6b140c6f3d519556e1b
TXN:  0xeabcfc2f4c9661051ac9cbf2b2ccf82390aefa5c143b4651d784e15c235e5d0d
TXN:  0xf9a818e04010e36f0058711b88dfa398b5724ec6819fb0b1577d632fa02bf62d
TXN:  0xee68af02eb97b2407898b12c3c9e15edef12b55d34fc0f6fef26aea947ac1a77
TXN:  0xcc3ef3c0f30c9f37f6af8e1593a8fa56f1d7481accd25fde5139910fc0dd41ee
TXN:  0xf1f4606edd2c0956e7cf311c279d999e7a15eef673dcc7d57393a5d4a8b04667
TXN:  0x225eb087b72b68895e573af74ac1e0484b542def4b5fd11a07f3a346e

,Wallet_Address,Tokens_Traded,Positive_Trades,Negative_Trades,Still_Holding_OR_Dusted,Sold_Incrementally_(NotJet),Transfer_To_Other_Account_OR_NULL,Transfer_Stablecoin,Average_Profit
1,0x0678a0cfc4b8e749033e7b452ff19f7bc90d5ad4,7,2,5,7,1,45,4,-14.464496
